In [ ]:
# Sumani
# https://www.linkedin.com/in/sumanaruban/
# https://github.com/Sumanaruban
# 29-7-2024

# Introduction

This notebook is a continuation of the previous Jupyter notebook [02_fully_connected_mnist.ipynb](./02_fully_connected_mnist.ipynb), where we built and trained a fully connected neural network to classify handwritten digits from the MNIST dataset using PyTorch. Having understood the fundamental steps involved in model development, this notebook provides a series of incremental exercises designed to deepen your understanding of various aspects of neural network training and optimization.

Each exercise introduces a small modification to the existing model or training process. These modifications will help you explore the effects of different model architectures, activation functions, optimizers, learning rates, regularization techniques, data augmentation, loss functions, and evaluation metrics. By completing these exercises, you will gain hands-on experience in tuning neural networks and improving model performance.

# How to Use This Notebook
1. **Review the Previous Notebook**: Ensure you are familiar with the steps and code in the previous notebook, as this notebook builds upon that foundation.
2. **Complete the Exercises**: Work through each exercise one by one. Make the specified changes to the model or training process and run the cells to see the effects.
3. **Document Your Observations**: For each exercise, take note of how the changes impact the model's training and evaluation metrics. Provide explanations for any improvements or deteriorations in performance.
4. **Experiment and Explore**: Feel free to experiment further by combining different techniques or exploring additional modifications beyond the provided exercises.

By the end of this notebook, you will have a deeper understanding of how various factors influence the training and performance of neural networks, equipping you with the knowledge and skills to effectively tune and optimize models for real-world applications.

In [1]:
# Import the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

In [2]:
# Define the transformation to apply to the images
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the images to tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize the pixel values with mean and std
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='.', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 58.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.70MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.53MB/s]


In [3]:
# Define the neural network model
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super(FullyConnectedNet, self).__init__()

        # First Layer
        self.layer_1 = nn.Linear(28*28, 512)
        self.activation_1 = nn.ReLU() # ReLU activation

        # Second Layer
        self.layer_2 = nn.Linear(512, 256)
        self.activation_2 = nn.ReLU() # ReLU activation

        # Third layer
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        # Flatten the image
        x = x.view(-1, 28*28)

        # Calling the first layer
        x = self.layer_1(x)
        x = self.activation_1(x)

        x = self.layer_2(x)
        x = self.activation_2(x)

        # Calling the second layer
        x = self.layer_3(x)  # Output layer
        return x

# Create an instance of the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FullyConnectedNet().to(device)
print(model)

FullyConnectedNet(
  (layer_1): Linear(in_features=784, out_features=512, bias=True)
  (activation_1): ReLU()
  (layer_2): Linear(in_features=512, out_features=256, bias=True)
  (activation_2): ReLU()
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)


In [4]:
from torchsummary import summary

summary(model, input_size=(1, 784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]         401,920
              ReLU-2                  [-1, 512]               0
            Linear-3                  [-1, 256]         131,328
              ReLU-4                  [-1, 256]               0
            Linear-5                   [-1, 10]           2,570
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.04
Estimated Total Size (MB): 2.06
----------------------------------------------------------------


In [5]:
!pip install torchviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [6]:
from torchviz import make_dot
import torch

x = torch.randn(1, 784)
y = model(x)

make_dot(y, params=dict(model.named_parameters())).render("nn_mnist_graph", format="png")

'nn_mnist_graph.png'

## Exercise 1: Modify the Model Architecture

    Add a Hidden Layer: Add an additional hidden layer with 256 neurons between the existing layers. Re-train the model and evaluate its performance.

# Exercise 2: Change the Activation Function

    Use Different Activation Functions: Replace the ReLU activation function with other activation functions such as Sigmoid or Tanh. Observe the impact on training and evaluation metrics.

# Exercise 3: Change the Optimizer

    Use Adam Optimizer: Replace the SGD optimizer with the Adam optimizer. Compare the training speed and final accuracy.

# Exercise 4: Adjust the Learning Rate

    Experiment with Learning Rates: Try different learning rates (e.g., 0.1, 0.001, 0.0001) with the SGD and Adam optimizers. Record the effects on the model's training performance.

# Exercise 5: Implement Dropout

    Add Dropout Layers: Introduce dropout layers with a dropout probability of 0.5 to the network. Check if the model's performance improves by reducing overfitting.

# Exercise 6: Batch Normalization

    Add Batch Normalization: Incorporate batch normalization layers into the network and observe any changes in training stability and speed.

# Exercise 7: Data Augmentation

    Apply Data Augmentation: Implement data augmentation techniques like random rotations, shifts, and flips to the training dataset. Evaluate the model's robustness to these transformations.

# Exercise 8: Learning Rate Scheduling

    Implement Learning Rate Scheduling: Use a learning rate scheduler to decrease the learning rate during training. Compare the training process and model performance.

# Exercise 9: Change the Loss Function

    Use Different Loss Functions: Experiment with different loss functions such as Mean Squared Error (MSE) and observe how the choice of loss function affects model training and performance.

# Exercise 10: Evaluate with Different Metrics

    Implement Precision, Recall, and F1-Score: Extend the evaluation metrics to include precision, recall, and F1-score. Analyze the model's performance using these metrics.

# Conclusion

These exercises will help you to understand the impact of different neural network architectures, optimization techniques, and evaluation metrics on the model's performance.